In [12]:
!pip install --upgrade pip
!pip install stim
!pip install qiskit

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [1]:
dir = "/Users/arya/Documents/Python/DATA"


In [7]:
import random
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

#number of qubits
n = 10
#number of layers
depth = 6
#the initial quantum circuit
qc= QuantumCircuit(n)
#Building a random quantum circuit
for i in range(depth):
    for j in range(n):
        gate = random.choice(['h', 'x', 'y', 'z', 's'])
        if gate == 'h':
            qc.h(j)
        elif gate == 'x':
            qc.x(j)
        elif gate == 'y':
            qc.y(j)
        elif gate == 'z':
            qc.z(j)
        elif gate == 's':
            qc.s(j)

    for q in range(n//2):
        q1, q2 = random.sample(range(n), 2)
        qc.cx(q1, q2)

print (qc)

#initial state
initial_state = Statevector.from_label("0" * n)
print ("Initial state:", initial_state)
final_state = initial_state.evolve(qc)
print ("Final state:", final_state)

     ┌───┐┌───┐                                             ┌───┐┌───┐┌───┐»
q_0: ┤ S ├┤ H ├──────────────────────────────────────────■──┤ X ├┤ S ├┤ Y ├»
     ├───┤├───┤┌───┐                                     │  └─┬─┘├───┤├───┤»
q_1: ┤ X ├┤ X ├┤ Y ├─────────────────────────────────────┼────┼──┤ X ├┤ Y ├»
     ├───┤└───┘├───┤┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐       │    │  └─┬─┘└───┘»
q_2: ┤ Y ├─────┤ X ├┤ H ├┤ X ├┤ X ├┤ H ├┤ Y ├┤ S ├───────┼────┼────┼───────»
     ├───┤     └─┬─┘├───┤└─┬─┘└───┘└───┘└───┘└───┘       │    │    │  ┌───┐»
q_3: ┤ X ├──■────■──┤ X ├──■──────────────■──────────────┼────■────┼──┤ H ├»
     ├───┤┌─┴─┐     └───┘┌───┐┌───┐┌───┐  │              │         │  └───┘»
q_4: ┤ Y ├┤ X ├──■───────┤ X ├┤ Y ├┤ H ├──┼──────────────┼─────────┼───────»
     ├───┤├───┤  │  ┌───┐└─┬─┘└───┘├───┤  │  ┌───┐       │         │       »
q_5: ┤ H ├┤ X ├──┼──┤ H ├──┼───────┤ X ├──┼──┤ S ├───────┼─────────┼───────»
     ├───┤└───┘┌─┴─┐└───┘  │  ┌───┐└─┬─┘  │  ├───┤┌───┐  │  ┌───┐  │       »

In [8]:
# stroring the final state in a text file
cl_file = dir + "/cl.txt"

with open(cl_file, "w") as f:
    for amp in final_state.data:
        real = amp.real
        imag = amp.imag
        f.write(f"{real:.6f} + {imag:.6f}im\n")


### Getting the cliffords operator Tableau form

In [11]:
from qiskit.quantum_info import Clifford

C = Clifford(qc)
C


Clifford(array([[False, False, False, False,  True, False, False, False,  True,
         True,  True, False, False, False, False, False, False,  True,
        False,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [ True, False,  True,  True,  True, False, False,  True,  True,
        False,  True, False,  True, False, False, False, False,  True,
        False, False,  True],
       [False, False,  True, False,  True, False, False, False,  True,
         True, False, False, False, False,  True,  True, False,  True,
         True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False, False, False,  True,  True,  True, False,  True, False,
        False,  True,  True, False, False, False, False, Fal

### Stabilizer Pauli Strings for zero state

In [5]:
import numpy as np
from qiskit.quantum_info import PauliList


def zero_state_stabilizers(n):
    #Creating all the binary combinations of length n
    combos = np.array(np.meshgrid(*[[0, 1]] * n)).T.reshape(-1, n)

    #Making the pauli_string equivalent
    stabilizers = []
    for c in combos:
        pauli_str = ''.join(['Z' if bit == 1 else 'I' for bit in c])
        stabilizers.append(pauli_str)

    return PauliList(stabilizers)
    

In [13]:
S = zero_state_stabilizers(10)

### Stabilizer Pauli Strings for our given state

In [14]:
P = S.evolve(C, frame='conjugate')

In [15]:
print(P.to_labels()[:10])
print("Total:", len(P))

['IIIIIIIIII', '-IIZXIXYYXZ', '-ZXYZXIZIIY', '-ZXXYXXXYXX', 'IXZYZIXYII', '-IXIZZXZIXZ', '-ZIXXYIYYIY', 'ZIYIYXIIXX', 'IIIIIXIIII', '-IIZXIIYYXZ']
Total: 1024


In [17]:
X = P.x.astype(int)
Z = P.z.astype(int)

np.savetxt("P_X_matrix.txt", X, fmt="%d")
np.savetxt("P_Z_matrix.txt", Z, fmt="%d")

print("Saved binary matrices for X and Z parts.")


Saved binary matrices for X and Z parts.
